In [4]:
import graphlab
import numpy as np

In [5]:
sales = graphlab.SFrame('kc_house_data.gl/')

2016-03-30 13:19:51,763 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1459358389.log


This non-commercial license of GraphLab Create is assigned to ashokkumar.h@husky.neu.edu and will expire on March 26, 2017. For commercial licensing options, visit https://dato.com/buy/.


In [6]:
def get_numpy(data_sframe, features, output):
    data_sframe['constant']=1
    features = ['constant']+ features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    output_sarray = data_sframe[output]
    output_sarray = output_sarray.to_numpy()
    return (feature_matrix, output_sarray)

In [7]:
(ex_features,ex_output)= get_numpy(sales, ['sqft_living'], ['price'])
print "features",ex_features[0,:]
print "output",ex_output[0]

features [  1.00000000e+00   1.18000000e+03]
output [ 221900.]


In [8]:
#Predicting output from regression weights
my_weights = np.array([1., 1.]) # the example weights
my_features = ex_features[0,]

def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix,weights)
    return predictions


In [9]:
#Testing the predictions 
test_pred = predict_output(ex_features,my_weights)
print test_pred

[ 1181.  2571.   771. ...,  1021.  1601.  1021.]


In [10]:
def feature_derivative(errors, feature):
    derivative = 2*np.dot(errors,feature)
    return derivative


In [11]:
#Testing the feature derivative. 
(example_features, example_output) = get_numpy(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_pred = predict_output(example_features, my_weights) 
errors = test_pred-example_output
feature = example_features[:,0]
derivative = feature_derivative(errors, feature)
print derivative
print -np.sum(example_output)*2

-23345850022.0
-23345850022.0


In [12]:
##GRADIENT DESCENT

In [21]:
from math import sqrt

def regression_gradient_descent(feature_matrix,output,initial_weights,step_size,tolerance):
    converged = False
    weights = np.array(initial_weights)
    while not converged:
        predictions = predict_output(feature_matrix, weights)
        errors = predictions - output
        gradient_sum_squares = 0
        for i in range(len(weights)):
            derivative = feature_derivative(errors, feature_matrix[:,i])
            gradient_sum_squares = gradient_sum_squares + (derivative * derivative)
            weights[i] = weights[i] - (step_size * derivative)
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [14]:
#split the data into training and test data.

train_data,test_data = sales.random_split(.8,seed=0)

In [19]:
simple_features = ['sqft_living']
my_output= 'price'
(simple_feature_matrix, output) = get_numpy(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [23]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size, tolerance)
print simple_weights
print round(simple_weights[1], 1)

[-46999.88716555    281.91211912]
281.9
